# Evalua que alumnos realizaron cada entrega, cuanto tiempo antes/despues de la fecha maxima de entrega, y la nota obtenida. Resultados se guardan en Drive.

In [ ]:
# Elejir si descargar
descargar = 'Si'



# Instalaciones
import requests
!pip install tika
from tika import parser
import pandas as pd
import random
import numpy as np
import time
from time import sleep
%pip install bs4
from bs4 import BeautifulSoup
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
options = webdriver.ChromeOptions() # Usamos chrome, se podria usar otro.
options.add_argument('--headless') # Chromium sin interfaz grafica
options.add_argument('--no-sandbox') # Seguridad
options.add_argument('--disable-dev-shm-usage') # configuracion de linux
options.add_argument('--user-agent=""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36""') # user agent
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from lxml import etree
!pip install tika
from tika import parser
from google.colab import files
from datetime import datetime
from datetime import timedelta
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC



# Instanciamos el webdriver
driver = webdriver.Chrome('chromedriver',options=options)

# Hacemos que vaya a la url
url = 'https://favaloro.tucampus.org/login/index.php'
driver.get(url)
sleep(1)

# Usuario
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="username"]'))).send_keys("41702703")

# Contraseña
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="password"]'))).send_keys("Cambio123*"+ Keys.ENTER)


# Link a cada comision
sleep(2)
links_comisiones = driver.find_elements("xpath", '//a[@href]')
links_comisiones = [link.get_attribute("href") for link in links_comisiones]
comisionesNombres = ['TV','C1','C2']
# Id's de las comisiones relevantes
ids = ['id=2025', 'id=2024', 'id=2163']
links_comisiones_final = []
for i,id in enumerate(ids):
  for link in links_comisiones:
    if id in link:
      links_comisiones_final.append(link) 
# Elimino duplicados
result = [] 
[result.append(x) for x in links_comisiones_final if x not in result] 
links_comisiones = result.copy()




# Iteracion por comision
nombresTodos = []
linksTodos = []
comisiones = []
condicionesCom = ['Satisfactorio', 'satisfactorio', 'supera lo','Supera lo','Supera Lo']
for i in range(len(links_comisiones)):
  driver.get(links_comisiones[i])
  sleep(2)
  links = driver.find_elements("xpath", "//div[@class='activityinstance']//a[@href]")
  links = [link.get_attribute("href") for link in links]
  
  # Nombre actividades
  nombres = driver.find_elements("xpath", "//a//span[@class='instancename']")
  nombres = [link.text for link in nombres]

  # Disponible o no para estudiantes
  vistos = driver.find_elements("xpath", '//ul[@class="section img-text"]//li')
  vistos = [link.text for link in vistos]

  # Ver si es una actividad y si esta vista por los estudiantes
  condiciones = ['Actividad','Autoevalua','Tarea']
  new_nombres = []
  new_links = []
  for i2,nombre in enumerate(nombres):
    for condicion in condiciones:
      if condicion in nombre:
        for visto in vistos:
          if nombre in visto:
            if 'No mostrado a' not in visto:
                new_nombres.append(nombre.split('\n')[0])
                new_links.append(links[i2])
                break
  nombresTodos.extend(new_nombres)     
  linksTodos.extend(new_links)
  nuevo = [comisionesNombres[i]] * len(new_links)
  comisiones.extend(nuevo)


# Iterar por link (tareas)
dfs = []
finalComision = []
contadorTV = 0
contadorC1 = 0
contadorC2 = 0
contador = 0
contadorViables = []

for iterador in range(len(linksTodos)):
  try:
                # TERCER INTENTO
    # Entrar a cada tarea
    sleep(1)
    driver.get(linksTodos[iterador])
    sleep(1)

    # Axceso a "ver intentos"
    driver.find_element("xpath", "//div[@class='quizattemptcounts']").click()
    sleep(1)

    # Obtener Clase
    fecha = driver.find_elements("xpath", "//li[@class='breadcrumb-item']//span")
    fecha = [f.text.split(' ')[0] for f in fecha if '/' in f.text]
    fecha = fecha[0]

    personasCh = driver.find_elements("xpath", "//tr[@class='gradedattempt']")
    personasCh = [i.text for i in personasCh]


    # Obtener nombre, estado tarea, tiempo y segunda forma de calificacion (comentarios). 
    nombreAlumno = []
    estadoTarea = []
    calificacion = []
    tiempo = []
    for persona in personasCh:
      if 'Seleccionar' in persona:
        dato = [i for i in persona.split('\n')]

        # Nombre
        nombreAlumno.append(dato[1])
        # Estado tarea
        estadoTarea.append('Enviado')

        # Retraso
        enviado = pd.to_datetime(' '.join(dato[-1].split('Finalizado ')[1].split(' ')[0:5]).replace('de ',''))
        fechaEntregaCh = pd.to_datetime(fecha.split('/')[1] + '-' + fecha.split('/')[0] + '-' + '2022')
        if comisiones[iterador] == 'TV':
          horas = 18
        else:
          horas = 8
        fechaEntregaCh = fechaEntregaCh + timedelta(days=7) + timedelta(hours=horas)
        retraso = enviado - fechaEntregaCh
        if str(retraso)[0] == '-':
          tiempo.append('A tiempo')
        else:
          retraso = str(retraso).split(' ')
          if retraso[0] == '0':
            retraso = retraso[-1].split(':')[0]
            final = ' horas despues'
          else:
            final = ' dias despues'
            retraso = retraso[0]
          retrasoFinal = retraso + final
          tiempo.append(retrasoFinal)

              # Calificacion
        contadorCali = 0
        for dat in dato[-1].split(' '):
          if ',' in dat:
            calificacion.append(dat)
            contadorCali += 1
            break
        if contadorCali == 0:
          calificacion.append('Sin dato')



    # Obtener comision
    contadorViables.append(comisiones[iterador])

    # Appendear a Df's
    dfs.append(pd.DataFrame(np.array([nombreAlumno,estadoTarea,tiempo,calificacion]).T, columns=['Nombre','Estado Tarea '+str(fecha)+' '+contadorViables[-1],
                                                                                                'Tiempo '+str(fecha)+' '+contadorViables[-1], 
                                                                                                'Calificacion '+str(fecha)+' '+contadorViables[-1]]))
    print(f'Exito unir a df de alumnos en itaracion {iterador} de {len(linksTodos)}')
      
    dfs1 = dfs.copy()
  except:

    try:
            # PRIMER INTENTO
      # Entrar a cada tarea
      sleep(1)
      driver.get(linksTodos[iterador])
      sleep(1)

      # Axceso a "ver mas"
      driver.find_element("xpath", "//a[@class='btn btn-secondary']").click()
      sleep(1)
      personas = driver.find_elements("xpath", "//tr[@class]")
      personas = [link.text for link in personas]

      # Obtener Clase
      fecha = driver.find_elements("xpath", "//li[@class='breadcrumb-item']//span")
      fecha = [f.text.split(' ')[0] for f in fecha if '/' in f.text]
      fecha = fecha[0]

      # Primera forma de obtener calificacion
      calificacion1 = []
      botonesCalificacion = driver.find_elements("xpath", "//td//a[@class='btn btn-primary']")
      botonesCalificacion = [bot.get_attribute('href') for bot in botonesCalificacion]
      for botonCali in botonesCalificacion:
        driver.get(botonCali)
        sleep(3)
        try:
          select = Select(WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="id_grade"]'))))
          element = select.first_selected_option
          calificacion1.append(element.text)
        except:
          calificacion1.append('')

      condicion = True
      condicion = [False for i in calificacion1 if i != '']

      # Volver a entrar a cada tarea
      sleep(1)
      driver.get(linksTodos[iterador])
      sleep(1)

      # Axceso a "ver mas"
      driver.find_element("xpath", "//a[@class='btn btn-secondary']").click()
      sleep(1)
      personas = driver.find_elements("xpath", "//tr[@class]")
      personas = [link.text for link in personas]


      # Acceso a comentarios
      botonesComentarios = driver.find_elements("xpath", "//i[@title='Comentarios']")
      for boton in botonesComentarios:
        sleep(1)
        boton.click()
      sleep(1)
      comentarios = driver.find_elements("xpath", '*//div[@class="text_to_html"]')
      comentarios = [link.text for link in comentarios]
      comentarios = [com for com in comentarios if [con for con in condicionesCom if con in com]]

      # Obtener nombre, estado tarea, tiempo y segunda forma de calificacion (comentarios). 
      nombreAlumno = []
      estadoTarea = []
      calificacion2 = []
      tiempo = []
      for persona in personas:
        if 'Seleccione' in persona:
          dato = [i for i in persona.split('\n')]
          nombreAlumno.append(dato[0].split('Seleccione ')[-1])
          replaceEstadoTarea = dato[2].replace('Borrador (no enviado)','Enviado')
          replaceEstadoTarea = replaceEstadoTarea.replace('Enviado para calificar','Enviado')
          estadoTarea.append(replaceEstadoTarea)

          # Retraso
          retrasoNoEnviado = [ire for ire in dato if 'La Tarea está retrasada por:' in ire]
          retrasoEnviado = [ire for ire in dato if 'después' in ire]
          if len(retrasoNoEnviado)>0:
            replaceTiempo = retrasoNoEnviado[0].replace('La Tarea está ','')
            tiempo.append(replaceTiempo)
          elif len(retrasoEnviado)>0:
            tiempo.append(retrasoEnviado[0])
          else:
            if replaceEstadoTarea != 'Sin entrega':
              tiempo.append('A tiempo')
            else:
              tiempo.append('')
          
          # Comentarios
          if dato[-2] != 'Comentarios (0)':
            datoJunto =  ''.join([str(item) for item in dato])
            if 'Comentarios (2)' in datoJunto and len(comentarios) > 0:
              comentarios.pop(0)
              calificacion2.append(comentarios.pop(0))
            elif 'Comentarios' in datoJunto and len(comentarios) > 0:
              calificacion2.append(comentarios.pop(0))
            else:
              calificacion2.append('')
          else:
            calificacion2.append('')
        if len(nombreAlumno) > len(calificacion2):
          nombreAlumno = nombreAlumno[0:len(calificacion2)]


      # Calificacion final
      new_calificacion = []
      for i in range(len(calificacion1)):
        if calificacion1[i] != '' and calificacion1[i] != 'No hay calificación':
          new_calificacion.append(calificacion1[i])
        elif calificacion2[i] != '':
          new_calificacion.append(calificacion2[i])
        else:
          new_calificacion.append('')
      calificacion = new_calificacion.copy()

      # Obtener comision
      contadorViables.append(comisiones[iterador])

      # Appendear a Df's
      dfs.append(pd.DataFrame(np.array([nombreAlumno,estadoTarea,tiempo,calificacion]).T, columns=['Nombre','Estado Tarea '+str(fecha)+' '+contadorViables[-1],
                                                                                                  'Tiempo '+str(fecha)+' '+contadorViables[-1], 
                                                                                                  'Calificacion '+str(fecha)+' '+contadorViables[-1]]))

      print(f'Exito unir a df de alumnos en itaracion {iterador} de {len(linksTodos)}')

    except:
      try:

          # SEGUNDO INTENTO
            # Entrar a cada tarea
        sleep(1)
        driver.get(linksTodos[iterador])
        sleep(1)

        # Acceso a datos
        datos = driver.find_elements("xpath", "//tbody//tr")
        datos = [link.text for link in datos]

        # Obtener Clase
        clase = driver.find_elements("xpath", "//li[@class='breadcrumb-item']//span")
        clase = [f.text.split(' ')[0] for f in clase if '/' in f.text]
        clase = clase[0]

        # Nombres
        nombres = [i.split('\n')[1] for i in datos]

        # Tiempo
        # Fecha de Entrega Tarea
        fechaEntrega = driver.find_elements("xpath", "//li[@class='breadcrumb-item']//span")
        fechaEntrega = [f.text.split(' ')[0] for f in fechaEntrega if '/' in f.text]
        fechaEntrega = '-'.join([fechaEntrega[0].split('/')[1], fechaEntrega[0].split('/')[0]])
        fechaEntrega = '2022-' + fechaEntrega
        fechaEntrega = pd.to_datetime(fechaEntrega)
        # Fecha Entrega Alumno
        fechaEntregaAlumno = [i.split('\n')[2] for i in datos]
        fechaEntregaAlumno = [pd.to_datetime(i) for i in fechaEntregaAlumno]
        retraso = []
        for fec in fechaEntregaAlumno:
          retraso.append(fechaEntrega - fec)
        retraso = [str(i) for i in retraso]
        retraso = [i.split(' ')[0] for i in retraso]
        retraso = [str(int(i) + 7) for i in retraso]
        newRetraso = []
        for iret in retraso:
          if iret[0] == '-':
            newRetraso.append(iret[1:] + ' días después')
          else:
            newRetraso.append('A tiempo')
        tiempo = newRetraso.copy()


        # Acceso a comentarios
        comentarios = []
        botonesComentarios = driver.find_elements("xpath", "//a[@class='w-100 h-100 d-block']")
        botonesComentarios = [link.get_attribute('href') for link in botonesComentarios]
        for boton in botonesComentarios:
          sleep(1)
          driver.get(boton)
          sleep(1)
          comentario = driver.find_elements("xpath", "//div[@class='text_to_html']")
          if len(comentario) > 0:
            comentarios.append(comentario[0].text)
          else:
            comentarios.append('')
        comentarios = [i.split('\n')[0] for i in comentarios]

        # Estado Tarea
        estadoTarea = ['Enviada' for i in range(len(comentarios))]

        # Obtener comision
        contadorViables.append(comisiones[iterador])

        # Appendear a Df's
        dfs.append(pd.DataFrame(np.array([nombres,estadoTarea,tiempo,comentarios]).T, columns=['Nombre','Estado Tarea '+str(clase)+' '+contadorViables[-1],
                                                                                                    'Tiempo Tarea '+str(clase)+' '+contadorViables[-1], 
                                                                                                    'Calificacion Tarea '+str(clase)+' '+contadorViables[-1]]))
        print(f'Exito unir a df de alumnos en itaracion {iterador} de {len(linksTodos)}')
      
      except:
        print(f'FRACASO get itaracion {iterador}')
        continue
dfs1 = dfs.copy()

# Eliminado de df's vacios
new_dfs1 = []
new_contadorViables = []
for i,v in enumerate(dfs1):
  if len(v) > 0:
    new_dfs1.append(v)
    new_contadorViables.append(contadorViables[i])
dfs1 = new_dfs1.copy()
contadorViables = new_contadorViables.copy()

# Creacion del df por comision
contadorTV = 0
contadorC1 = 0
contadorC2 = 0

for i,v in enumerate(dfs1):
  v = v.set_index('Nombre')
  if contadorViables[i] == 'TV':
    contadorTV += 1
    if contadorTV > 1:
      try:
        dfTV = pd.merge(dfTV,v,on='Nombre',how='outer')
      except:
        continue
    else:
      dfTV = v
  elif contadorViables[i] == 'C1':
    contadorC1 += 1
    if contadorC1 > 1:
      try:
        dfC1 = pd.merge(dfC1,v,on='Nombre',how='outer')
      except:
        continue
    else:
      dfC1 = v
  elif contadorViables[i] == 'C2':
    contadorC2 += 1
    if contadorC2 > 1:
      try:
        dfC2 = pd.merge(dfC2,v,on='Nombre',how='outer')
      except:
        continue
    else:
      dfC2 = v

# arreglo de las columnas con titulos _x e _y + quedarse con la que tenga mas datos
# quedarse con la que tenga mas datos
dfsCom = [dfTV, dfC1, dfC2]
for i1,dfCom in enumerate(dfsCom):
  columnasTodas = []
  columnasNombre = []
  columnasX = [col for col in dfCom.columns if '_x' in col]
  for col in columnasX:
    columnasTodas = [col, col[:-2], col[:-2]+'_y']
    columnasNombre = [c for c in columnasTodas if c in dfCom.columns]
    for i2,x2 in enumerate(dfCom[columnasNombre[0]].values):
      if x2 == '':
        for cn in columnasNombre[1:]:
          if dfCom[cn].values[i2] != '':
            dfsCom[i1][columnasNombre[0]].values[i2] = dfCom[cn].values[i2]
    for cn in columnasNombre[1:]:
      dfsCom[i1].drop(cn, inplace=True, axis=1)

# arreglo de las columnas con titulos _x e _y
dfsCom = [dfTV, dfC1, dfC2]
for i1,dfCom in enumerate(dfsCom):
  columnasTodas = [col for col in dfCom.columns if '_x' in col or '_y' in col]
  for col in columnasTodas:
    dfsCom[i1].rename(columns={col: col[:-2]}, inplace=True)

# Orden alfabetico
dfTV.sort_values('Nombre',inplace=True)
dfC1.sort_values('Nombre',inplace=True)
dfC2.sort_values('Nombre',inplace=True)

# Correccion de Nan's
dfsFinales = [dfTV, dfC1, dfC2]
for idf in range(len(dfsFinales)):
  for col in dfsFinales[idf].columns:
    if 'Estado' in col:
      dfsFinales[idf][col].fillna('Sin entrega', inplace=True)
    elif 'Tiempo ' in col:
      dfsFinales[idf][col].fillna('', inplace=True)
    elif 'Calificacion' in col:
      dfsFinales[idf][col].fillna('', inplace=True)

# pasarlo a excel
with pd.ExcelWriter("Autoevaluaciones_Personalidad.xlsx") as writer:
    dfTV.to_excel(writer, sheet_name="TV")  
    dfC1.to_excel(writer, sheet_name="C1")  
    dfC2.to_excel(writer, sheet_name="C2")  

# Descargar
if descargar == 'Si':
  files.download('/content/Autoevaluaciones_Personalidad.xlsx')

AttributeError: ignored